In [44]:
import pandas as pd
import requests
data = pd.read_csv("C:\\Users\\aviel\\Python\\Homework\\destinations_LP_crawler_Ex5.csv",encoding = 'ISO-8859-1')
print(data)

         city                           city_LP
0     Antalya       mediterranean-coast/antalya
1      Athens                            Athens
2   Ayia-napa  the-republic-of-cyprus/Agia-napa
3     Bangkok                           Bangkok
4   Barcelona                         Barcelona
..        ...                               ...
71     Venice                            Venice
72     Vienna                            Vienna
73     Warsaw                            Warsaw
74  Zakynthos                         Zakynthos
75     Zurich                            Zurich

[76 rows x 2 columns]


In [45]:

L = list()

In [46]:
for i in data['city']:
    address = i
    if  address == 'Krakï¿' :
        address = 'krakow'
    api_key = "INSERT KEY"

    url="https://maps.googleapis.com/maps/api/geocode/json?address=%s&key=%s" % (address,api_key)
   
    try:
        response = requests.get(url)
        if not response.status_code == 200:
            print("HTTP error",response.status_code)
        else:
                try:
                    response_data = response.json()
                    x = response_data['results'][0]['address_components']
                    y=(x[-1])
                    try:   
                        COUNTRY = int(y['long_name'] )
                        w = x[-2]
                        country = w['long_name']
                        L.append(country)
                    except:    
                        country = (y['long_name'] )
                        if country == '846 00' :
                            country = 'Greece'
                        L.append(country)               
                   
                except:
                    print("Response not in valid JSON format")
    except:
        print("Something went wrong with requests.get")

for i in range(len(L)):
    if L[i]=='United States':
        L[i] ='usa'
    
    
data['country'] = L

In [47]:
info_list = list()
LP_destantions = pd.DataFrame()  
for i,j in zip(data['country']  , data['city_LP']):
           
          
            link = "https://www.lonelyplanet.com/" + i +"/" + j 
            from bs4 import BeautifulSoup          
            try:
                response = requests.get(link)
                if not response.status_code == 200:
                    info_list.append("SOMETHING got wrong with the LP_destantions")
                    print(link)
                result_page = BeautifulSoup(response.content,'html.parser')  
                for words in result_page.find_all('div',class_= "readMore_content___5EuR"):
                    info_list.append(words.get_text().replace('\n',' '))             
            except:
                "error"
                
                print( "eroorr" )
LP_destantions['city']=data['city']
LP_destantions['country']= L            
LP_destantions['Description']  = info_list              

print(LP_destantions)

LP_destantions.to_csv( "C:\\Users\\aviel\\Python\\Homework\\ex3_text.txtLP_destinations.csv',index=False")


         city      country                                        Description
0     Antalya       Turkey  Once seen simply as the gateway to the Turkish...
1      Athens       Greece  With equal measures of grunge and grace, Athen...
2   Ayia-napa       Cyprus  Endless blue-sky days and a coastline riddled ...
3     Bangkok     Thailand  Same same, but different. This Thailish T-shir...
4   Barcelona        Spain  Barcelona is an enchanting seaside city with b...
..        ...          ...                                                ...
71     Venice        Italy  Imagine the audacity of building a city of mar...
72     Vienna      Austria  Baroque streetscapes and imperial palaces set ...
73     Warsaw       Poland  A phoenix arisen from the ashes, Poland's capi...
74  Zakynthos       Greece  Zakynthos, also known by its Italian name Zant...
75     Zurich  Switzerland  Culturally vibrant, efficiently run and attrac...

[76 rows x 3 columns]
